# BQ Monitor Merge Cost Analysis

**Purpose**: Calculate the annual cost of BQ Monitor merge operations

**Formula**:
```
Total Cost = (BigQuery Reservation API × % merge jobs) + Storage Costs + Cloud Pub/Sub Costs
```

**Components**:
1. BigQuery Reservation API cost × % merge jobs (from traffic_classification)
2. Storage costs on monitor-base-us-prod
3. Cloud Pub/Sub costs for monitor-base-us-prod

**Date**: 2025-11-06


In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Initialize BigQuery client
client = bigquery.Client(project='narvar-data-lake')

print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


## 1. Load and Analyze DoIT CSV Data


In [ ]:
# Load the CSV file
csv_path = '/Users/cezarmihaila/Downloads/BQ Detailed 01 monthly.csv'
df = pd.read_csv(csv_path)

# Display basic info
print(f"Total rows: {len(df)}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
df.head()


### 1.1 Calculate BigQuery Reservation API Costs


In [ ]:
# Filter for BigQuery Reservation API costs for bq-narvar-admin
bq_reservation_df = df[
    (df['Service'] == 'BigQuery Reservation API') & 
    (df['Project/Account ID'] == 'bq-narvar-admin')
]

print("BigQuery Reservation API SKUs:")
print(bq_reservation_df[['SKU', 'Service']].to_string(index=False))
print("\n" + "="*80)

# Get month columns (exclude metadata columns)
month_cols = [col for col in df.columns if col not in ['Value', 'Service', 'SKU', 'Project/Account ID']]
print(f"\nMonth columns: {month_cols}")

# Calculate annual cost for each SKU
bq_reservation_annual = {}
for idx, row in bq_reservation_df.iterrows():
    sku = row['SKU']
    # Sum all months, handling empty/NaN values
    annual_cost = sum([float(row[col]) if pd.notna(row[col]) and row[col] != '' else 0 for col in month_cols])
    bq_reservation_annual[sku] = annual_cost
    print(f"\n{sku}:")
    print(f"  Annual Cost: ${annual_cost:,.2f}")

total_bq_reservation = sum(bq_reservation_annual.values())
print("\n" + "="*80)
print(f"\nTOTAL BigQuery Reservation API (Annual): ${total_bq_reservation:,.2f}")


### 1.2 Calculate Storage Costs for monitor-base-us-prod


In [ ]:
# Filter for BigQuery storage costs on monitor-base-us-prod
storage_df = df[
    (df['Service'] == 'BigQuery') & 
    (df['Project/Account ID'] == 'monitor-base-us-prod') &
    (df['SKU'].str.contains('Storage', case=False, na=False))
]

print("BigQuery Storage SKUs for monitor-base-us-prod:")
print(storage_df[['SKU', 'Service', 'Project/Account ID']].to_string(index=False))
print("\n" + "="*80)

# Calculate annual cost for each storage SKU
storage_annual = {}
for idx, row in storage_df.iterrows():
    sku = row['SKU']
    annual_cost = sum([float(row[col]) if pd.notna(row[col]) and row[col] != '' else 0 for col in month_cols])
    storage_annual[sku] = annual_cost
    print(f"\n{sku}:")
    print(f"  Annual Cost: ${annual_cost:,.2f}")

total_storage = sum(storage_annual.values())
print("\n" + "="*80)
print(f"\nTOTAL Storage Costs (monitor-base-us-prod, Annual): ${total_storage:,.2f}")


### 1.3 Calculate Cloud Pub/Sub Costs for monitor-base-us-prod


In [ ]:
# Filter for Cloud Pub/Sub costs on monitor-base-us-prod
pubsub_df = df[
    (df['Service'] == 'Cloud Pub/Sub') & 
    (df['Project/Account ID'] == 'monitor-base-us-prod')
]

print("Cloud Pub/Sub SKUs for monitor-base-us-prod:")
print(pubsub_df[['SKU', 'Service', 'Project/Account ID']].to_string(index=False))
print("\n" + "="*80)

# Calculate annual cost for each Pub/Sub SKU
pubsub_annual = {}
for idx, row in pubsub_df.iterrows():
    sku = row['SKU']
    annual_cost = sum([float(row[col]) if pd.notna(row[col]) and row[col] != '' else 0 for col in month_cols])
    pubsub_annual[sku] = annual_cost
    print(f"\n{sku}:")
    print(f"  Annual Cost: ${annual_cost:,.2f}")

total_pubsub = sum(pubsub_annual.values())
print("\n" + "="*80)
print(f"\nTOTAL Cloud Pub/Sub Costs (monitor-base-us-prod, Annual): ${total_pubsub:,.2f}")


## 2. Query BigQuery to Find Merge Job Percentage

**Scope**: AUTOMATED category merge operations writing to monitor projects (like monitor-base-us-prod)

**Analysis Period**: Sep-Oct 2024 (baseline), extrapolated to annual costs


In [ ]:
# Query to find AUTOMATED merge jobs writing to monitor projects
query_merge_analysis = """
DECLARE start_date DATE DEFAULT '2024-09-01';
DECLARE end_date DATE DEFAULT '2024-10-31';

WITH traffic_data AS (
  SELECT
    job_id,
    job_type,
    consumer_category,
    consumer_subcategory,
    project_id,
    query_text_sample,
    total_slot_ms,
    execution_time_seconds,
    estimated_slot_cost_usd,
    -- Identify MERGE operations writing to monitor projects
    CASE 
      WHEN (UPPER(query_text_sample) LIKE '%MERGE%INTO%' 
            OR UPPER(query_text_sample) LIKE '%MERGE INTO%')
        AND REGEXP_CONTAINS(UPPER(query_text_sample), r'MERGE\\s+INTO\\s+[`\\[]?monitor-[a-z0-9]+-us-[a-z]+')
        THEN TRUE
      ELSE FALSE
    END AS is_monitor_merge_job
  FROM `narvar-data-lake.query_opt.traffic_classification`
  WHERE DATE(start_time) BETWEEN start_date AND end_date
    AND total_slot_ms IS NOT NULL
    AND consumer_category = 'AUTOMATED'  -- Only AUTOMATED category
)

SELECT
  -- Overall AUTOMATED metrics
  COUNT(*) AS total_automated_jobs,
  COUNTIF(is_monitor_merge_job) AS monitor_merge_jobs,
  ROUND(COUNTIF(is_monitor_merge_job) / COUNT(*) * 100, 2) AS monitor_merge_job_pct,
  
  -- Slot consumption (AUTOMATED only)
  SUM(total_slot_ms) AS total_automated_slot_ms,
  SUM(CASE WHEN is_monitor_merge_job THEN total_slot_ms ELSE 0 END) AS monitor_merge_slot_ms,
  ROUND(SUM(CASE WHEN is_monitor_merge_job THEN total_slot_ms ELSE 0 END) / SUM(total_slot_ms) * 100, 2) AS monitor_merge_slot_pct,
  
  -- Cost metrics (AUTOMATED only)
  ROUND(SUM(estimated_slot_cost_usd), 2) AS total_automated_cost_usd,
  ROUND(SUM(CASE WHEN is_monitor_merge_job THEN estimated_slot_cost_usd ELSE 0 END), 2) AS monitor_merge_cost_usd,
  ROUND(SUM(CASE WHEN is_monitor_merge_job THEN estimated_slot_cost_usd ELSE 0 END) / SUM(estimated_slot_cost_usd) * 100, 2) AS monitor_merge_cost_pct,
  
  -- Execution time
  ROUND(SUM(execution_time_seconds) / 3600, 2) AS total_automated_exec_hours,
  ROUND(SUM(CASE WHEN is_monitor_merge_job THEN execution_time_seconds ELSE 0 END) / 3600, 2) AS monitor_merge_exec_hours
  
FROM traffic_data;
"""

print("Querying traffic_classification for AUTOMATED merge jobs to monitor projects...")
print("Analysis Period: Sep-Oct 2024 baseline")
print("\nQuery cost estimate: ~5-10GB\n")
print("="*80)

# Run the query
merge_analysis_df = client.query(query_merge_analysis).to_dataframe()
merge_analysis_df


In [ ]:
# Extract monitor merge percentages
monitor_merge_jobs = merge_analysis_df['monitor_merge_jobs'].iloc[0]
total_automated_jobs = merge_analysis_df['total_automated_jobs'].iloc[0]
monitor_merge_slot_pct = merge_analysis_df['monitor_merge_slot_pct'].iloc[0]
monitor_merge_cost_usd = merge_analysis_df['monitor_merge_cost_usd'].iloc[0]

print("Monitor Merge Job Metrics (Sep-Oct 2024 Baseline - AUTOMATED Category Only):")
print("="*80)
print(f"Monitor Merge Jobs: {monitor_merge_jobs:,} out of {total_automated_jobs:,} AUTOMATED jobs ({merge_analysis_df['monitor_merge_job_pct'].iloc[0]}%)")
print(f"Monitor Merge Slot %: {monitor_merge_slot_pct}% of AUTOMATED slots")
print(f"Monitor Merge Cost %: {merge_analysis_df['monitor_merge_cost_pct'].iloc[0]}% of AUTOMATED costs")
print(f"\nMonitor Merge Execution Time: {merge_analysis_df['monitor_merge_exec_hours'].iloc[0]:,.2f} hours")
print(f"Monitor Merge Cost (2-month baseline): ${monitor_merge_cost_usd:,.2f}")
print(f"Total AUTOMATED Cost (2-month baseline): ${merge_analysis_df['total_automated_cost_usd'].iloc[0]:,.2f}")


### 2.1 Alternative: Sample Monitor Merge Queries (for validation)


In [ ]:
# Validate the regex pattern by sampling merge queries
query_sample_monitor_merges = """
DECLARE start_date DATE DEFAULT '2024-09-01';
DECLARE end_date DATE DEFAULT '2024-10-31';

SELECT
  project_id,
  consumer_category,
  consumer_subcategory,
  ROUND(total_slot_ms / 3600000, 4) AS slot_hours,
  estimated_slot_cost_usd,
  query_text_sample
FROM `narvar-data-lake.query_opt.traffic_classification`
WHERE DATE(start_time) BETWEEN start_date AND end_date
  AND total_slot_ms IS NOT NULL
  AND consumer_category = 'AUTOMATED'
  AND (
    UPPER(query_text_sample) LIKE '%MERGE%INTO%' 
    OR UPPER(query_text_sample) LIKE '%MERGE INTO%'
  )
  AND REGEXP_CONTAINS(UPPER(query_text_sample), r'MERGE\\s+INTO\\s+[`\\[]?monitor-[a-z0-9]+-us-[a-z]+')
ORDER BY slot_hours DESC
LIMIT 10;
"""

print("Sampling AUTOMATED merge queries to monitor projects (top 10 by slot hours)...")
print("This helps validate the regex pattern is working correctly.\n")

sample_monitor_merges_df = client.query(query_sample_monitor_merges).to_dataframe()

if len(sample_monitor_merges_df) > 0:
    print(f"✓ Found {len(sample_monitor_merges_df)} sample queries")
    print("\nSample queries:")
    for idx, row in sample_monitor_merges_df.iterrows():
        print(f"\n{idx+1}. Project: {row['project_id']}, Subcategory: {row['consumer_subcategory']}")
        print(f"   Slot Hours: {row['slot_hours']:.4f}, Cost: ${row['estimated_slot_cost_usd']:.4f}")
        print(f"   Query: {row['query_text_sample'][:150]}...")
else:
    print("⚠ No queries matched the pattern. The regex may need adjustment.")
    print("Consider broadening the search to all MERGE operations in AUTOMATED category.")


## 3. Calculate Total Annual BQ Monitor Merge Cost


In [ ]:
# Calculate annual costs from 2-month baseline period
# Extrapolation factor: 12 months / 2 months = 6
extrapolation_factor = 6

# Monitor merge reservation cost (annualized)
# Using monitor_merge_slot_pct from AUTOMATED category
monitor_merge_reservation_cost = total_bq_reservation * (monitor_merge_slot_pct / 100)

print("="*80)
print("ANNUAL BQ MONITOR MERGE COST CALCULATION")
print("="*80)
print("\nBASELINE PERIOD: Sep-Oct 2024 (2 months)")
print(f"Extrapolation Factor: {extrapolation_factor}x (to annual)")

print("\n1. BigQuery Reservation API Cost (Monitor Merge Portion - AUTOMATED):")
print(f"   Total Reservation Cost (Annual):           ${total_bq_reservation:>15,.2f}")
print(f"   Monitor Merge Slot % (of AUTOMATED):       {monitor_merge_slot_pct:>15.2f}%")
print(f"   Monitor Merge Reservation Cost (Annual):   ${monitor_merge_reservation_cost:>15,.2f}")

print("\n2. Storage Costs (monitor-base-us-prod):")
print(f"   Annual Storage Cost:                       ${total_storage:>15,.2f}")

print("\n3. Cloud Pub/Sub Costs (monitor-base-us-prod):")
print(f"   Annual Pub/Sub Cost:                       ${total_pubsub:>15,.2f}")

print("\n" + "="*80)
total_annual_cost = monitor_merge_reservation_cost + total_storage + total_pubsub
print(f"TOTAL ANNUAL BQ MONITOR MERGE COST:           ${total_annual_cost:>15,.2f}")
print("="*80)

print("\n\nCost Breakdown:")
print(f"  Compute (Monitor Merge):  ${monitor_merge_reservation_cost:>12,.2f}  ({monitor_merge_reservation_cost/total_annual_cost*100:>5.1f}%)")
print(f"  Storage:                  ${total_storage:>12,.2f}  ({total_storage/total_annual_cost*100:>5.1f}%)")
print(f"  Pub/Sub:                  ${total_pubsub:>12,.2f}  ({total_pubsub/total_annual_cost*100:>5.1f}%)")
print(f"  {'─'*60}")
print(f"  TOTAL:                    ${total_annual_cost:>12,.2f}  (100.0%)")


## 4. Summary and Export


In [ ]:
# Create summary dataframe
summary_data = {
    'Cost Component': [
        'BigQuery Reservation (AUTOMATED Monitor Merge)',
        'Storage (monitor-base-us-prod)',
        'Cloud Pub/Sub (monitor-base-us-prod)',
        'TOTAL'
    ],
    'Annual Cost (USD)': [
        monitor_merge_reservation_cost,
        total_storage,
        total_pubsub,
        total_annual_cost
    ],
    'Percentage': [
        f"{monitor_merge_reservation_cost/total_annual_cost*100:.1f}%",
        f"{total_storage/total_annual_cost*100:.1f}%",
        f"{total_pubsub/total_annual_cost*100:.1f}%",
        "100.0%"
    ],
    'Notes': [
        f'Based on {monitor_merge_slot_pct:.2f}% of AUTOMATED slots (Sep-Oct 2024)',
        'All storage SKUs for monitor-base-us-prod',
        'All Pub/Sub operations for monitor-base-us-prod',
        'Total cost to offset'
    ]
}

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Save to CSV
output_path = '/Users/cezarmihaila/workspace/do_it_query_optimization_queries/bigquery-optimization-queries/narvar/monitor_merge_cost_summary.csv'
summary_df.to_csv(output_path, index=False)
print(f"\n\nSummary saved to: {output_path}")

print("\n" + "="*80)
print("ANSWER TO QUESTION:")
print("="*80)
print(f"\nThe BQ Monitor merge (AUTOMATED category, writing to monitor projects)")
print(f"is costing us ${total_annual_cost:,.2f} annually.")
print(f"\nThis cost would need to be offset by any alternative solution.")
print("\n" + "="*80)

# Additional context
print("\n\nKEY ASSUMPTIONS:")
print("─" * 80)
print(f"1. Scope: AUTOMATED merge jobs writing to monitor-* projects")
print(f"2. Baseline: Sep-Oct 2024 (2 months)")
print(f"3. Monitor Merge Slot %: {monitor_merge_slot_pct:.2f}% of AUTOMATED category slots")
print(f"4. Storage: All storage costs for monitor-base-us-prod")
print(f"5. Pub/Sub: All Pub/Sub operations for monitor-base-us-prod")
print(f"6. Reservation API: Annual cost across all commitment types")
